In [17]:
# Input data

import scipy.io as spio
import numpy as np

new_executive = spio.loadmat(file_name='../data/bci_EEG_data-new/new_executive.mat')
new_executive_data = new_executive['new_executive']
print(new_executive_data.shape, new_executive_data.dtype)

new_executive_data_transposed = new_executive_data.transpose(2, 0, 1)
print(new_executive_data_transposed.shape)

# Deviding the input data into train and validation
# For creating the training and testing set, 30% percent of each subject is considered as test and
# 70% of each subject is conidered as training.
length = int(new_executive_data_transposed.shape[0] * 0.30)
# length

train_data_all = new_executive_data_transposed[:-length]
test_data = new_executive_data_transposed[-length:]

print(new_executive_data_transposed.shape, new_executive_data_transposed.dtype, 
 train_data_all.shape, train_data_all.dtype, 
 test_data.shape, test_data.dtype)

# 30% of the total training data is validation,
# 70% of the total training data is training
# This is applied to every single subject data.
length2 = int(train_data_all.shape[0] * 0.30)
# length2

train_data = train_data_all[:-length2]
valid_data = train_data_all[-length2:]

print(train_data_all.shape, train_data_all.dtype, 
 train_data.shape, train_data.dtype, 
 valid_data.shape, valid_data.dtype)

(3072, 13, 1380) float64
(1380, 3072, 13)
(1380, 3072, 13) float64 (966, 3072, 13) float64 (414, 3072, 13) float64
(966, 3072, 13) float64 (677, 3072, 13) float64 (289, 3072, 13) float64


In [18]:
new_executive_label = spio.loadmat(file_name='../data/bci_EEG_data-new/new_executive_label.mat')
# new_executive_label.keys()
new_executive_label_all = new_executive_label['new_executive_label']
print(new_executive_label_all.shape, new_executive_label_all.dtype)
# new_executive_label_all.max(axis=0)+1

label_train_all = new_executive_label_all[:-length]
label_test = new_executive_label_all[-length:]

print(new_executive_label_all.shape, new_executive_label_all.dtype,
 label_test.shape, label_test.dtype, 
 label_train_all.shape, label_train_all.dtype)

label_train = label_train_all[:-length2]
label_valid = label_train_all[-length2:]
print(label_train.shape, label_valid.shape)

(1380, 1) uint16
(1380, 1) uint16 (414, 1) uint16 (966, 1) uint16
(677, 1) (289, 1)


In [19]:
# # Normalizing input data
# def normalize(inputs, inputs_all):
#     return (inputs - inputs_all.mean(axis=0)[None,:,:]) / inputs_all.std(axis=0)[None,:,:]
# Yalda suggested this normalization.
def normalize(inputs):
    return (inputs - inputs.mean(axis=0)[None,:,:]) / inputs.std(axis=0)[None,:,:]

# onehot vectorizing output labels
def one_hot(labels, n_class):
    """ One-hot encoding """
    expansion = np.eye(n_class)
    y = expansion[:, labels-1].T
    assert y.shape[1] == n_class, "Wrong number of labels!"

    return y

# get minibatches for learning
def get_batches(X, y, batch_size):
    """ Return a generator for batches """
    n_batches = len(X) // batch_size
    X, y = X[:n_batches*batch_size], y[:n_batches*batch_size]

    # Loop over batches and yield
    for b in range(0, len(X), batch_size):
        yield X[b:b+batch_size], y[b:b+batch_size]

# Standardize/normalize train and test
# X_train_norm_all = normalize(inputs=FacesDataTrain, inputs_all=FacesDataAll)
X_train_norm = normalize(inputs=train_data)
X_valid_norm = normalize(inputs=valid_data)
X_test_norm = normalize(inputs=test_data)

print(X_train_norm.shape, X_train_norm.dtype, 
X_valid_norm.shape, X_valid_norm.dtype,
X_test_norm.shape, X_test_norm.dtype)

(677, 3072, 13) float64 (289, 3072, 13) float64 (414, 3072, 13) float64


In [20]:
### Hyperparameters

# Input data
batch_size = X_train_norm.shape[0]// 100 # minibatch size & number of minibatches
seq_len = X_train_norm.shape[1] # Number of steps: each trial length
n_channels = X_train_norm.shape[2] # number of channels in each trial
print('batch_size, seq_len, n_channels', batch_size, seq_len, n_channels)

# Output labels
n_classes = int(new_executive_label_all.max(axis=0) + 1)
print('n_classes', n_classes)

# Tweekable parameters
learning_rate = 0.001 #1e-3
epochs = 100 # num iterations for updating model
keep_prob = 0.50 # 90% neurons are kept and 10% are dropped out

batch_size, seq_len, n_channels 6 3072 13
n_classes 4


In [21]:
Y_train = np.array(label_train, dtype=int).reshape(-1)
Y_valid = np.array(label_valid, dtype=int).reshape(-1)
Y_test = np.array(label_test, dtype=int).reshape(-1)

Y_train_onehot = one_hot(labels=Y_train, n_class=n_classes)
Y_valid_onehot = one_hot(labels=Y_valid, n_class=n_classes)
Y_test_onehot = one_hot(labels=Y_test, n_class=n_classes)

print(Y_train_onehot.shape, Y_valid_onehot.shape, Y_test_onehot.shape, 
 X_train_norm.shape, X_valid_norm.shape, X_test_norm.shape)

print(Y_train_onehot.dtype, Y_valid_onehot.dtype, Y_test_onehot.dtype,
 X_train_norm.dtype, X_valid_norm.dtype, X_test_norm.dtype)

(677, 4) (289, 4) (414, 4) (677, 3072, 13) (289, 3072, 13) (414, 3072, 13)
float64 float64 float64 float64 float64 float64


In [22]:
# GPUs or CPU

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

#  No graphs is needed on tensorflow
inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs_')
labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels_')
keep_prob_ = tf.placeholder(tf.float32, name = 'keep_prob_')
learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate_')# Construct the LSTM inputs and LSTM cells

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


In [23]:
# Forward pass: Convolutional Layers, FC Layer, and Output layer
# 677, 3072, 13
# (batch, 3072, 13) --> (batch, 1534, 26)
# (3072 - 6 + 0)/2 + 1 = (3066/2)+1= 1533 +1 = 1534
# 2/6 with strides/kernel_size is 33.333% non-overlap/diff region and 
# 66.666% overlapping window/ common region
in_conv = inputs_
out_conv = tf.layers.conv1d(inputs=in_conv, filters=26, kernel_size=6, strides=2, padding='valid')
out_conv = tf.layers.batch_normalization(inputs=out_conv)
out_conv = tf.nn.relu(features=out_conv)
out_conv = tf.nn.dropout(x=out_conv, keep_prob=keep_prob_)
print(in_conv.shape, out_conv.shape)

# To reduce the size for memory efficiency & equivariancy/invariency/ minicolumns
# (batch, 1534, 26) --> (batch, 511, 26)
# (1534 - 4 + 0)/3 + 1 = (1530/3)+1= 510 +1 = 511
# 3/4 with strides/kernel_size is 
# 75% non-overlap/diff region and 
# 25% overlapping window/ common region
in_pool = out_conv
out_pool = tf.layers.max_pooling1d(inputs=in_pool, pool_size=4, strides=3, padding='valid')
print(in_pool.shape, out_pool.shape)

# (batch, 511, 26) --> (batch, 254, 52)
# (511 - 5 + 0)/2 + 1 = (506/2)+1= 253+1= 254
# 2/5 with strides/kernel_size is 40% non-overlap/diff region and 
# 60% overlapping window/ common region
in_conv = out_pool
out_conv = tf.layers.conv1d(inputs=in_conv, filters=52, kernel_size=5, strides=2, padding='valid')
out_conv = tf.layers.batch_normalization(inputs=out_conv)
out_conv = tf.nn.relu(features=out_conv)
out_conv = tf.nn.dropout(x=out_conv, keep_prob=keep_prob_)
print(in_conv.shape, out_conv.shape)

# To reduce the size for memory efficiency & equivariancy/invariency/ minicolumns
# (batch, 254, 52) --> (batch, 25, 52)
# (254 - 14 + 0)/10 + 1 = (240/10)+1= 24 +1 = 25
# 10/14 with strides/kernel_size is 
# ~66.666% non-overlap/diff region and 
# ~33.33% overlapping window/ common region
in_pool = out_conv
out_pool = tf.layers.max_pooling1d(inputs=in_pool, pool_size=14, strides=10, padding='valid')
print(in_pool.shape, out_pool.shape)

# (batch, 25, 52) --> (batch, 25*52) --> (batch, 25*52*2)
in_fc = tf.reshape(tensor=out_pool, shape=(-1, 25*52))
out_fc = tf.layers.dense(inputs=in_fc, units=25*52*2)
out_fc = tf.layers.batch_normalization(inputs=out_fc)
out_fc = tf.nn.relu(features=out_fc)
out_fc = tf.nn.dropout(x=out_fc, keep_prob=keep_prob_)
print(out_pool.shape, in_fc.shape, out_fc.shape)

# (batch, 25*52*2) --> (batch, 4) 
logits = tf.layers.dense(inputs=out_fc, units=n_classes)
print(out_fc.shape, logits.shape)

(?, 3072, 13) (?, 1534, 26)
(?, 1534, 26) (?, 511, 26)
(?, 511, 26) (?, 254, 52)
(?, 254, 52) (?, 25, 52)
(?, 25, 52) (?, 1300) (?, 2600)
(?, 2600) (?, 4)


In [24]:
# Backward pass: error backpropagation
# Cost function
cost_tensor = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_)
cost = tf.reduce_mean(input_tensor=cost_tensor)

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate_).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [ ]:
# Train the network
# Plotting the acc and loss curve
train_acc = []
train_loss = []

valid_acc = []
valid_loss = []

with tf.Session() as sess:
    # initalize session global variables just in the case they are initialized.
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs):
       
        # Loop over batches
        for x, y in get_batches(X_train_norm, Y_train_onehot, batch_size):
            
            # Feed dictionary
            feed = {inputs_:x, labels_:y, keep_prob_: keep_prob, learning_rate_:learning_rate}
            loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
            
            train_acc.append(acc)
            train_loss.append(loss)
            
            ################################ Validation
            # Initialize 
            loss_v_batch, acc_v_batch = [], []

            # Loop over batches
            for x_v, y_v in get_batches(X_valid_norm, Y_valid_onehot, batch_size):

                # Feed dictionary
                feed = {inputs_:x_v, labels_:y_v, keep_prob_: 1.0}
                loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)
                
                acc_v_batch.append(acc_v)
                loss_v_batch.append(loss_v)
                
            valid_acc.append(np.mean(acc_v_batch))
            valid_loss.append(np.mean(loss_v_batch))
            
            # Print info
            print("Epoch: {}/{}".format(e+1, epochs),
                  "Train loss: {:6f}".format(loss),
                  "Valid loss: {:.6f}".format(np.mean(valid_loss)),
                  "Train acc: {:6f}".format(acc),
                  "Valid acc: {:.6f}".format(np.mean(valid_acc)))
            
    ################################ Test
    # Initialize 
    acc_batch, loss_batch = [], []

    # Loop over batches
    for x, y in get_batches(X_test_norm, Y_test_onehot, batch_size):

        # Feed dictionary
        feed = {inputs_:x, labels_:y, keep_prob_:1.0}
        loss, acc = sess.run([cost, accuracy], feed_dict = feed)

        acc_batch.append(acc)
        loss_batch.append(loss)

    # Print info
    print("Epoch: {}/{}".format(e+1, epochs),
          "Test loss: {:6f}".format(np.mean(loss_batch)),
          "Test acc: {:6f}".format(np.mean(acc_batch)))

Epoch: 1/100 Train loss: 2.203544 Valid loss: 6.618005 Train acc: 0.500000 Valid acc: 0.333333
Epoch: 1/100 Train loss: 18.787941 Valid loss: 8.374361 Train acc: 0.333333 Valid acc: 0.335069
Epoch: 1/100 Train loss: 26.265539 Valid loss: 7.579428 Train acc: 0.333333 Valid acc: 0.335648
Epoch: 1/100 Train loss: 16.928116 Valid loss: 6.268805 Train acc: 0.333333 Valid acc: 0.334201
Epoch: 1/100 Train loss: 5.343141 Valid loss: 5.509328 Train acc: 0.333333 Valid acc: 0.333333
Epoch: 1/100 Train loss: 5.706849 Valid loss: 4.999148 Train acc: 0.500000 Valid acc: 0.333333
Epoch: 1/100 Train loss: 6.963777 Valid loss: 4.648093 Train acc: 0.333333 Valid acc: 0.333333
Epoch: 1/100 Train loss: 6.694564 Valid loss: 4.266350 Train acc: 0.333333 Valid acc: 0.333333
Epoch: 1/100 Train loss: 5.057148 Valid loss: 3.927539 Train acc: 0.166667 Valid acc: 0.335648
Epoch: 1/100 Train loss: 3.432205 Valid loss: 3.698458 Train acc: 0.333333 Valid acc: 0.335764
Epoch: 1/100 Train loss: 3.469756 Valid loss: 3

Epoch: 1/100 Train loss: 1.291860 Valid loss: 1.555349 Train acc: 0.166667 Valid acc: 0.331617
Epoch: 1/100 Train loss: 1.043879 Valid loss: 1.551854 Train acc: 0.500000 Valid acc: 0.331636
Epoch: 1/100 Train loss: 1.115499 Valid loss: 1.548414 Train acc: 0.166667 Valid acc: 0.331654
Epoch: 1/100 Train loss: 1.033396 Valid loss: 1.545031 Train acc: 0.500000 Valid acc: 0.331673
Epoch: 1/100 Train loss: 1.199141 Valid loss: 1.541704 Train acc: 0.333333 Valid acc: 0.331691
Epoch: 1/100 Train loss: 1.186422 Valid loss: 1.538439 Train acc: 0.166667 Valid acc: 0.331597
Epoch: 1/100 Train loss: 1.189693 Valid loss: 1.535241 Train acc: 0.000000 Valid acc: 0.331579
Epoch: 1/100 Train loss: 1.092583 Valid loss: 1.532092 Train acc: 0.166667 Valid acc: 0.331670
Epoch: 1/100 Train loss: 1.164690 Valid loss: 1.528999 Train acc: 0.166667 Valid acc: 0.331830
Epoch: 1/100 Train loss: 1.119220 Valid loss: 1.525962 Train acc: 0.333333 Valid acc: 0.331916
Epoch: 1/100 Train loss: 1.127659 Valid loss: 1.52

Epoch: 2/100 Train loss: 1.003829 Valid loss: 1.380244 Train acc: 0.833333 Valid acc: 0.336982
Epoch: 2/100 Train loss: 1.003823 Valid loss: 1.379108 Train acc: 0.666667 Valid acc: 0.336923
Epoch: 2/100 Train loss: 1.090404 Valid loss: 1.377974 Train acc: 0.500000 Valid acc: 0.336980
Epoch: 2/100 Train loss: 1.109234 Valid loss: 1.376840 Train acc: 0.166667 Valid acc: 0.337076
Epoch: 2/100 Train loss: 1.051993 Valid loss: 1.375705 Train acc: 0.500000 Valid acc: 0.337132
Epoch: 2/100 Train loss: 1.256275 Valid loss: 1.374571 Train acc: 0.000000 Valid acc: 0.337187
Epoch: 2/100 Train loss: 1.216211 Valid loss: 1.373437 Train acc: 0.333333 Valid acc: 0.337223
Epoch: 2/100 Train loss: 1.016390 Valid loss: 1.372300 Train acc: 0.500000 Valid acc: 0.337145
Epoch: 2/100 Train loss: 1.126986 Valid loss: 1.371155 Train acc: 0.500000 Valid acc: 0.337068
Epoch: 2/100 Train loss: 1.082573 Valid loss: 1.370004 Train acc: 0.500000 Valid acc: 0.337048
Epoch: 2/100 Train loss: 1.101710 Valid loss: 1.36

Epoch: 3/100 Train loss: 1.537331 Valid loss: 1.303600 Train acc: 0.500000 Valid acc: 0.333451
Epoch: 3/100 Train loss: 1.044156 Valid loss: 1.302900 Train acc: 0.333333 Valid acc: 0.333503
Epoch: 3/100 Train loss: 1.043397 Valid loss: 1.302211 Train acc: 0.500000 Valid acc: 0.333554
Epoch: 3/100 Train loss: 0.951883 Valid loss: 1.301534 Train acc: 0.500000 Valid acc: 0.333605
Epoch: 3/100 Train loss: 1.163813 Valid loss: 1.300865 Train acc: 0.333333 Valid acc: 0.333669
Epoch: 3/100 Train loss: 1.157602 Valid loss: 1.300205 Train acc: 0.666667 Valid acc: 0.333745
Epoch: 3/100 Train loss: 1.200066 Valid loss: 1.299552 Train acc: 0.333333 Valid acc: 0.333756
Epoch: 3/100 Train loss: 1.226288 Valid loss: 1.298909 Train acc: 0.333333 Valid acc: 0.333767
Epoch: 3/100 Train loss: 1.222672 Valid loss: 1.298275 Train acc: 0.166667 Valid acc: 0.333842
Epoch: 3/100 Train loss: 1.289497 Valid loss: 1.297651 Train acc: 0.166667 Valid acc: 0.333878
Epoch: 3/100 Train loss: 0.964355 Valid loss: 1.29

Epoch: 4/100 Train loss: 1.143146 Valid loss: 1.259501 Train acc: 0.333333 Valid acc: 0.332603
Epoch: 4/100 Train loss: 1.008258 Valid loss: 1.259074 Train acc: 0.666667 Valid acc: 0.332714
Epoch: 4/100 Train loss: 1.176270 Valid loss: 1.258649 Train acc: 0.000000 Valid acc: 0.332872
Epoch: 4/100 Train loss: 1.150053 Valid loss: 1.258226 Train acc: 0.500000 Valid acc: 0.333040
Epoch: 4/100 Train loss: 1.113410 Valid loss: 1.257806 Train acc: 0.666667 Valid acc: 0.333216
Epoch: 4/100 Train loss: 1.103943 Valid loss: 1.257388 Train acc: 0.333333 Valid acc: 0.333363
Epoch: 4/100 Train loss: 0.957118 Valid loss: 1.256973 Train acc: 0.666667 Valid acc: 0.333537
Epoch: 4/100 Train loss: 0.992546 Valid loss: 1.256558 Train acc: 0.500000 Valid acc: 0.333672
Epoch: 4/100 Train loss: 1.454016 Valid loss: 1.256145 Train acc: 0.500000 Valid acc: 0.333806
Epoch: 4/100 Train loss: 1.170152 Valid loss: 1.255733 Train acc: 0.166667 Valid acc: 0.333891
Epoch: 4/100 Train loss: 1.393461 Valid loss: 1.25

...

Epoch: 96/100 Train loss: 0.541207 Valid loss: 1.068911 Train acc: 0.833333 Valid acc: 0.395629
Epoch: 96/100 Train loss: 0.923184 Valid loss: 1.068908 Train acc: 0.500000 Valid acc: 0.395633
Epoch: 96/100 Train loss: 0.392700 Valid loss: 1.068906 Train acc: 0.833333 Valid acc: 0.395636
Epoch: 96/100 Train loss: 1.610559 Valid loss: 1.068903 Train acc: 0.833333 Valid acc: 0.395639
Epoch: 96/100 Train loss: 0.942768 Valid loss: 1.068901 Train acc: 0.500000 Valid acc: 0.395643
Epoch: 96/100 Train loss: 1.160013 Valid loss: 1.068899 Train acc: 0.333333 Valid acc: 0.395646
Epoch: 96/100 Train loss: 0.498106 Valid loss: 1.068896 Train acc: 0.833333 Valid acc: 0.395650
Epoch: 96/100 Train loss: 0.622486 Valid loss: 1.068894 Train acc: 0.666667 Valid acc: 0.395653
Epoch: 96/100 Train loss: 0.612963 Valid loss: 1.068892 Train acc: 0.666667 Valid acc: 0.395657
Epoch: 96/100 Train loss: 0.985928 Valid loss: 1.068890 Train acc: 0.500000 Valid acc: 0.395660
Epoch: 96/100 Train loss: 0.848476 Valid

Epoch: 96/100 Train loss: 0.948863 Valid loss: 1.068765 Train acc: 0.500000 Valid acc: 0.395854
Epoch: 96/100 Train loss: 0.884300 Valid loss: 1.068767 Train acc: 0.666667 Valid acc: 0.395855
Epoch: 96/100 Train loss: 0.599853 Valid loss: 1.068769 Train acc: 0.666667 Valid acc: 0.395857
Epoch: 96/100 Train loss: 0.823556 Valid loss: 1.068770 Train acc: 0.333333 Valid acc: 0.395858
Epoch: 96/100 Train loss: 1.123117 Valid loss: 1.068771 Train acc: 0.500000 Valid acc: 0.395860
Epoch: 96/100 Train loss: 0.591769 Valid loss: 1.068772 Train acc: 0.666667 Valid acc: 0.395861
Epoch: 96/100 Train loss: 0.860041 Valid loss: 1.068773 Train acc: 0.333333 Valid acc: 0.395864
Epoch: 96/100 Train loss: 1.090742 Valid loss: 1.068774 Train acc: 0.333333 Valid acc: 0.395866
Epoch: 96/100 Train loss: 0.920521 Valid loss: 1.068774 Train acc: 0.666667 Valid acc: 0.395868
Epoch: 96/100 Train loss: 1.037791 Valid loss: 1.068775 Train acc: 0.500000 Valid acc: 0.395870
Epoch: 96/100 Train loss: 0.853093 Valid

Epoch: 97/100 Train loss: 0.650806 Valid loss: 1.068531 Train acc: 0.666667 Valid acc: 0.396163
Epoch: 97/100 Train loss: 0.980823 Valid loss: 1.068529 Train acc: 0.666667 Valid acc: 0.396167
Epoch: 97/100 Train loss: 0.515490 Valid loss: 1.068528 Train acc: 0.833333 Valid acc: 0.396171
Epoch: 97/100 Train loss: 0.672779 Valid loss: 1.068530 Train acc: 0.833333 Valid acc: 0.396175
Epoch: 97/100 Train loss: 0.469876 Valid loss: 1.068538 Train acc: 0.666667 Valid acc: 0.396175
Epoch: 97/100 Train loss: 0.471873 Valid loss: 1.068557 Train acc: 0.500000 Valid acc: 0.396173
Epoch: 97/100 Train loss: 1.702276 Valid loss: 1.068568 Train acc: 0.500000 Valid acc: 0.396172
Epoch: 97/100 Train loss: 0.553800 Valid loss: 1.068572 Train acc: 0.666667 Valid acc: 0.396172
Epoch: 97/100 Train loss: 0.143487 Valid loss: 1.068574 Train acc: 0.833333 Valid acc: 0.396171
Epoch: 97/100 Train loss: 0.119337 Valid loss: 1.068574 Train acc: 1.000000 Valid acc: 0.396171
Epoch: 97/100 Train loss: 1.369767 Valid

Epoch: 98/100 Train loss: 0.963755 Valid loss: 1.068474 Train acc: 0.500000 Valid acc: 0.396263
Epoch: 98/100 Train loss: 1.007922 Valid loss: 1.068472 Train acc: 0.666667 Valid acc: 0.396265
Epoch: 98/100 Train loss: 1.075382 Valid loss: 1.068469 Train acc: 0.166667 Valid acc: 0.396269
Epoch: 98/100 Train loss: 1.911347 Valid loss: 1.068467 Train acc: 0.833333 Valid acc: 0.396271
Epoch: 98/100 Train loss: 0.277254 Valid loss: 1.068465 Train acc: 0.833333 Valid acc: 0.396273
Epoch: 98/100 Train loss: 0.038847 Valid loss: 1.068463 Train acc: 1.000000 Valid acc: 0.396275
Epoch: 98/100 Train loss: 0.271609 Valid loss: 1.068461 Train acc: 0.833333 Valid acc: 0.396277
Epoch: 98/100 Train loss: 0.311026 Valid loss: 1.068459 Train acc: 0.833333 Valid acc: 0.396277
Epoch: 98/100 Train loss: 0.536212 Valid loss: 1.068457 Train acc: 0.666667 Valid acc: 0.396277
Epoch: 98/100 Train loss: 0.370679 Valid loss: 1.068455 Train acc: 0.833333 Valid acc: 0.396277
Epoch: 98/100 Train loss: 0.077228 Valid

Epoch: 99/100 Train loss: 0.507025 Valid loss: 1.068408 Train acc: 0.833333 Valid acc: 0.396209
Epoch: 99/100 Train loss: 0.220290 Valid loss: 1.068410 Train acc: 1.000000 Valid acc: 0.396205
Epoch: 99/100 Train loss: 0.550062 Valid loss: 1.068413 Train acc: 0.833333 Valid acc: 0.396202
Epoch: 99/100 Train loss: 0.381732 Valid loss: 1.068417 Train acc: 0.833333 Valid acc: 0.396198
Epoch: 99/100 Train loss: 0.499582 Valid loss: 1.068420 Train acc: 0.666667 Valid acc: 0.396194
Epoch: 99/100 Train loss: 2.022507 Valid loss: 1.068424 Train acc: 0.500000 Valid acc: 0.396189
Epoch: 99/100 Train loss: 0.645468 Valid loss: 1.068427 Train acc: 0.666667 Valid acc: 0.396183
Epoch: 99/100 Train loss: 0.436594 Valid loss: 1.068431 Train acc: 0.666667 Valid acc: 0.396180
Epoch: 99/100 Train loss: 0.480186 Valid loss: 1.068434 Train acc: 0.666667 Valid acc: 0.396177
Epoch: 99/100 Train loss: 1.111853 Valid loss: 1.068437 Train acc: 0.333333 Valid acc: 0.396176
Epoch: 99/100 Train loss: 1.155431 Valid

In [ ]:
import matplotlib.pyplot as mplot

mplot.plot(train_loss, label='train_loss')
mplot.plot(valid_loss, label='valid_loss')
mplot.legend()
mplot.show()

In [ ]:
# import matplotlib.pyplot as mplot

mplot.plot(train_acc, label='train_acc')
mplot.plot(valid_acc, label='valid_acc')
mplot.legend()
mplot.show()